<a href="https://colab.research.google.com/github/mortezaaghajanzadeh/Machine-learning-in-Finance/blob/main/Lecture%203/introduction_to_tree_based_models_lecture_3_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lecture 3: Tree-Based Models.**
### Example: train bagging, boosting, and random forest models to predict historical house price growth using the MacroHistory database: https://www.macrohistory.net/database/.

In [ ]:
# Install packages.
!pip install shap

In [ ]:
# Load packages.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Load data.
file_path = 'https://www.dropbox.com/scl/fi/3kcanu4zfvr7it0wiqnb3/JSTdatasetR3.xlsx?rlkey=an6g1cqxicz0005g4fp1ggv21&dl=1'
data = pd.read_excel(file_path, sheet_name='Data')

## **Transform variables.**

In [ ]:
# Business loans to GDP ratio.
data['tbus_to_gdp'] = data['tbus'] / data['gdp']

# Household loans to GDP ratio.
data['thh_to_gdp'] = data['thh'] / data['gdp']

# Mortgage loans to non-financial firms in private sector to GDP ratio.
data['tmort_to_gdp'] = data['tmort'] / data['gdp']

# Total loans to non-financial firms in private sector to GDP ratio.
data['tloans_to_gdp'] = data['tloans'] / data['gdp']

# Net exports to GDP ratio.
data['net_exports_to_gdp'] = (data['exports'] - data['imports']) / data['gdp']

# Government surplus to GDP ratio.
data['surplus_to_gdp'] = (data['revenue'] - data['expenditure']) / data['gdp']

# Calculate Inflation (Percentage change in price index)
data['stock_price_growth'] = data.groupby('country')['stocks'].pct_change() * 100

# Calculate Inflation (Percentage change in price index)
data['inflation'] = data.groupby('country')['cpi'].pct_change() * 100

# Calculate House Price Growth (Percentage change in house prices)
data['hpnom'] = data.groupby('country')['hpnom'].pct_change() * 100

# Shift crisis variable ahead one period.
data['hpnom_growth_lead'] = data.groupby('country')['hpnom'].shift(-1)

# Drop missing values.
data.dropna(inplace=True)

# Drop data prior to 1970.
data = data[data['year'] >= 1970].copy()

## **Visualize features and target.**

In [ ]:
# Plot house price growth.
sns.histplot(data['hpnom_growth_lead'], kde=True)
plt.title('Distribution of House Price Growth')
plt.xlabel('House Price Growth (%)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Plotting house price growth time series for selected countries.
sample_countries = data['country'].unique()[0:10]
for country in sample_countries:
    subset = data[data['country'] == country]
    plt.plot(subset['year'], subset['hpnom_growth_lead'], label=country)

# Set plot labels and legend.
plt.title('House Price Growth Over Time by Country')
plt.xlabel('Year')
plt.ylabel('House Price Growth (%)')
plt.legend()
plt.show()

## **Define features and target.**

In [ ]:
# Define features.
X = data.drop(columns=['crisisJST',
                       'country',
                       'ifs',
                       'iso',
                       'year',
                       'pop',
                       'rgdpmad',
                       'rgdppc',
                       'rconpc',
                       'gdp',
                       'iy',
                       'cpi',
                       'stocks',
                       'ca',
                       'imports',
                       'exports',
                       'narrowm',
                       'money',
                       'revenue',
                       'expenditure',
                       'xrusd',
                       'tloans',
                       'tmort',
                       'thh',
                       'tbus',
                       'hpnom',
                       'hpnom_growth_lead'])

# Define target.
y = data['hpnom_growth_lead']

## **Train models.**

In [ ]:
# Generate training and test sets.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=103)

In [ ]:
# Instantiate random forest model.
rf_model = RandomForestRegressor(n_estimators=102,
                                 random_state=103)

# Train random forest.
rf_model.fit(X_train, y_train)

In [ ]:
# Instantiate gradient boosting model.
gbt_model = GradientBoostingRegressor(n_estimators=100,
                                      learning_rate=0.1,
                                      random_state=103)

# Train gradient boosting model.
gbt_model.fit(X_train, y_train)

In [ ]:
# Instantiate bagging model.
bag_model = BaggingRegressor(n_estimators=100,
                             random_state=102)

# Train bagging model.
bag_model.fit(X_train, y_train)

## **Visualize training process.**

In [ ]:
# Store MSE from each step.
test_score = [mean_squared_error(y_test, y_pred) for y_pred in gbt_model.staged_predict(X_test)]

# Plot error as function of trees.
plt.plot(test_score)
plt.title('Gradient Boosting Model Error')
plt.xlabel('Number of Trees')
plt.ylabel('Mean Squared Error')
plt.show()

## **Evaluate models.**

In [ ]:
# Define function to evaluate model performance.
def evaluate_regression_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, mae, r2

# Evaluate model.
rf_mse, rf_mae, rf_r2 = evaluate_regression_model(rf_model, X_test, y_test)
gbt_mse, gbt_mae, gbt_r2 = evaluate_regression_model(gbt_model, X_test, y_test)
bag_mse, bag_mae, bag_r2 = evaluate_regression_model(bag_model, X_test, y_test)

# Print evaluation metrics.
print("Random Forest Regressor - MSE:", rf_mse, "MAE:", rf_mae, "R2:", rf_r2)
print("Gradient Boosting Regressor - MSE:", gbt_mse, "MAE:", gbt_mae, "R2:", gbt_r2)
print("Bagging Regressor - MSE:", bag_mse, "MAE:", bag_mae, "R2:", bag_r2)

## **Tuning models.**

#### Bagging.

In [ ]:
# Set hyperparameters.
param_set_1 = {'n_estimators': 100, 'max_samples': 1.0, 'max_features': 1.0}
param_set_2 = {'n_estimators': 150, 'max_samples': 0.8, 'max_features': 0.8}

# Instantiate models.
bag_model_1 = BaggingRegressor(**param_set_1, random_state=103)
bag_model_2 = BaggingRegressor(**param_set_2, random_state=103)

# Train models.
bag_model_1.fit(X_train, y_train)
bag_model_2.fit(X_train, y_train)

### Random forest.

In [ ]:
# Set hyperparameters.
param_set_1 = {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1}
param_set_2 = {'n_estimators': 150, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 2}

# Instantiate models.
rf_model_1 = RandomForestRegressor(**param_set_1, random_state=103)
rf_model_2 = RandomForestRegressor(**param_set_2, random_state=103)

# Train models.
rf_model_1.fit(X_train, y_train)
rf_model_2.fit(X_train, y_train)

#### Gradient boosting.

In [ ]:
# Set hyperparameters.
param_set_1 = {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3}
param_set_2 = {'n_estimators': 150, 'learning_rate': 0.05, 'max_depth': 5}

# Instantiate models.
gbt_model_1 = GradientBoostingRegressor(**param_set_1, random_state=103)
gbt_model_2 = GradientBoostingRegressor(**param_set_2, random_state=103)

# Train models.
gbt_model_1.fit(X_train, y_train)
gbt_model_2.fit(X_train, y_train)

## **Interpret feature importance using Shapley values.**

In [ ]:
# Instantiate a SHAP explainer.
explainer = shap.TreeExplainer(gbt_model)

# Calculate SHAP values.
shap_values = explainer.shap_values(X_train)

In [ ]:
# Plot summary of Shapley values.
shap.summary_plot(shap_values, X_train, feature_names=X_train.columns)

In [ ]:
# Examine model output's dependence on interest rates.
shap.dependence_plot('ltrate', shap_values, X_train)